NMS(Non-Max Supression)-非极大值抑制
===

# 1.NMS
## 1.1.NMS介绍
在目标检测中，常会利用非极大值抑制算法(NMS，non maximum suppression)对生成的大量候选框进行后处理，去除冗余的候选框，得到最佳检测框，以加快目标检测的效率

其本质思想是其思想是搜素局部最大值，抑制非极大值，如下图
![image](Images/03/02/03_02_001.jpg)
每个选出来的Bounding Box检测框用$x,y,h,w, confidence score，Pdog,Pcat$表示，confidence score表示background和foreground的置信度得分，取值范围$[0,1]$。Pdog,Pcat分布代表类别是狗和猫的概率。如果是100类的目标检测模型，BBox输出向量为5+100=105

## 1.2.NMS算法过程
NMS主要就是通过迭代的形式，不断的以最大得分的框去与其他框做IoU操作，并过滤那些IoU较大（即交集较大）的框。
![image](Images/03/02/03_02_002.jpg)
![image](Images/03/02/03_02_003.jpg)
1. 根据候选框的类别分类概率做排序，假如有4个BBox，其置信度A>B>C>D
2. 先标记最大概率矩形框A是算法要保留的BBox
3. 从最大概率矩形框A开始，分别判断ABC与D的重叠度IOU（两框的交并比）是否大于某个设定的阈值(0.5)，假设D与A的重叠度超过阈值，那么就舍弃D
4. 从剩下的矩形框BC中，选择概率最大的B，标记为保留，然后判读C与B的重叠度，扔掉重叠度超过设定阈值的矩形框
5. 一直重复进行，标记完所有要保留下来的矩形框

- 如果是two stage算法，通常在选出BBox有BBox位置(x,y,h,w)和confidence score，没有类别的概率。因为程序是生成BBox，再将选择的BBox的feature map做rescale(一般用ROI pooling)，然后再用分类器分类。NMS一般只能在CPU计算，这也是two stage相对耗时的原因
- 但如果是one stage作法，BBox有位置信息(x,y,h,w)、confidence score，以及类别概率，相对于two stage少了后面的rescale和分类程序，所以计算量相对少。

# 1.3.NMS计算

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
FIG_SIZE = (30,20)

In [ ]:
#NMS implementation in Python and Numpy
def nms_python(bboxes,psocres,threshold):
    '''
    NMS: first sort the bboxes by scores , 
        keep the bbox with highest score as reference,
        iterate through all other bboxes, 
        calculate Intersection Over Union (IOU) between reference bbox and other bbox
        if iou is greater than threshold,then discard the bbox and continue.
        
    Input:
        bboxes(numpy array of tuples) : Bounding Box Proposals in the format (x_min,y_min,x_max,y_max).
        pscores(numpy array of floats) : confidance scores for each bbox in bboxes.
        threshold(float): Overlapping threshold above which proposals will be discarded.
        
    Output:
        filtered_bboxes(numpy array) :selected bboxes for which IOU is less than threshold. 
    '''
    #Unstacking Bounding Box Coordinates
    bboxes = bboxes.astype('float')
    x_min = bboxes[:,0]
    y_min = bboxes[:,1]
    x_max = bboxes[:,2]
    y_max = bboxes[:,3]
    
    #Sorting the pscores in descending order and keeping respective indices.
    sorted_idx = psocres.argsort()[::-1]
    #Calculating areas of all bboxes.Adding 1 to the side values to avoid zero area bboxes.
    bbox_areas = (x_max-x_min+1)*(y_max-y_min+1)
    
    #list to keep filtered bboxes.
    filtered = []
    while len(sorted_idx) > 0:
        #Keeping highest pscore bbox as reference.
        rbbox_i = sorted_idx[0]
        #Appending the reference bbox index to filtered list.
        filtered.append(rbbox_i)
        
        #Calculating (xmin,ymin,xmax,ymax) coordinates of all bboxes w.r.t to reference bbox
        overlap_xmins = np.maximum(x_min[rbbox_i],x_min[sorted_idx[1:]])
        overlap_ymins = np.maximum(y_min[rbbox_i],y_min[sorted_idx[1:]])
        overlap_xmaxs = np.minimum(x_max[rbbox_i],x_max[sorted_idx[1:]])
        overlap_ymaxs = np.minimum(y_max[rbbox_i],y_max[sorted_idx[1:]])
        
        #Calculating overlap bbox widths,heights and there by areas.
        overlap_widths = np.maximum(0,(overlap_xmaxs-overlap_xmins+1))
        overlap_heights = np.maximum(0,(overlap_ymaxs-overlap_ymins+1))
        overlap_areas = overlap_widths*overlap_heights
        
        #Calculating IOUs for all bboxes except reference bbox
        ious = overlap_areas/(bbox_areas[rbbox_i]+bbox_areas[sorted_idx[1:]]-overlap_areas)
        
        #select indices for which IOU is greather than threshold
        delete_idx = np.where(ious > threshold)[0]+1
        delete_idx = np.concatenate(([0],delete_idx))
        
        #delete the above indices
        sorted_idx = np.delete(sorted_idx,delete_idx)
        
    
    #Return filtered bboxes
    return bboxes[filtered].astype('int')

In [ ]:
#Reading image and copying for furthur use.
sampleimage = cv2.imread("Images/03/02/03_02_004.jpg")
sampleimage = cv2.cvtColor(sampleimage,cv2.COLOR_BGR2RGB)
sampleimageallbb = sampleimage.copy()
sampleimagenmsbb = sampleimage.copy()

In [ ]:
#Sample BBoxes and corresponding scores.
bboxes = np.array([(165,127,296,455),(148,142,257,459),(142,137,270,465),(129,122,302,471),
                   (327,262,604,465),(349,253,618,456),(369,248,601,470)])
pscores = np.array([0.8,0.95,0.81,0.85,0.94,0.83,0.82])
#Drawing all rectangular bboxes on original image
for bbox in bboxes:
    top_left = bbox[0],bbox[1]
    bottom_right = bbox[2],bbox[3]
    cv2.rectangle(sampleimageallbb,top_left, bottom_right,(255, 0, 0), 2)
#Getting nms filtered bboxes
bboxes_after_nms = nms_python(bboxes,pscores,0.3)

#Drawing nms filtered rectangular bboxes on original image
for bbox in bboxes_after_nms:
    top_left = bbox[0],bbox[1]
    bottom_right = bbox[2],bbox[3]
    cv2.rectangle(sampleimagenmsbb,top_left, bottom_right,(255, 0, 0), 2)
    
image_list = [sampleimageallbb,sampleimagenmsbb]
titles = ["BBoxes before NMS","BBoxes after NMS"]
    
fig, axes = plt.subplots(1, 2,figsize=FIG_SIZE)
for axis,(image,title) in zip(axes,zip(image_list,titles)):
    axis.imshow(image)
    axis.axis('off')
    axis.set_title(title)

## 1.4.缺点分析
1. NMS算法中的最大问题就是它将相邻检测框的分数均强制归零(既将重叠部分大于重叠阈值Nt的检测框移除)。在这种情况下，如果一个真实物体在重叠区域出现，则将导致对该物体的检测失败并降低了算法的平均检测率(average precision, AP)。
2. NMS的阈值也不太容易确定，设置过小会出现误删，设置过高又容易增大误检
3. NMS一般只能使用CPU计算，无法使用GPU计算

# 2.Soft-NMS
